# 简介

经过opencvtest 01 & 02 & 03的初步探究，目前确定的图书图像分割方案为：
1. Canny边缘提取
2. 以膨胀4倍，腐蚀8倍
3. 切出最大图书区域
4. 找到图书边缘
5. 切割图书

## 01 Canny边缘检测

In [ ]:
import cv2
import numpy as np
import deal_border as db
pathname = '../../data/one/'
filename = 'img12'
src = cv2.imread(pathname + filename + '.jpg')
blurred = cv2.GaussianBlur(src,(3,3),0)
gray=cv2.cvtColor(blurred,cv2.COLOR_RGB2GRAY)
edge_output=cv2.Canny(gray,50,100)
g_h,g_w = edge_output.shape
#cv2.imshow('src',src)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

## 02 膨胀腐蚀

In [ ]:
kernel1 = np.ones((4,4),np.uint8)
kernel2 = np.ones((8,8),np.uint8)
dilation = cv2.dilate(edge_output,kernel1,iterations=1)
erosion1 = cv2.erode(dilation,kernel2,iterations=1)
#cv2.imshow('dilation',dilation)
#cv2.imshow('erosion',erosion1)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

## 03 找出图书主体区域

1. 先找出x方向上，y轴中的ROI，取最大
2. 在此ROI中再找到y方向，x轴上的ROI，取两边

In [ ]:
y1,y2 = db.getROI_y(erosion1)
y1,y2 = db.Merge2ROI(y1,y2)
index = db.findMaxROI(y1,y2)
x1,x2 = db.getROI_x(erosion1[y1[index]:y2[index]])
src_main = src[y1[index]:y2[index],x1[0]:x2[-1]]
edge_main = edge_output[y1[index]:y2[index],x1[0]:x2[-1]]
m_h,m_w = edge_main.shape
cv2.imshow('src_main',src_main)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 04 找到图书的边缘

In [ ]:
import find_border as fb
line_lists = fb.detect_bookedge(edge_main)
new_list = fb.del_restline(fb.get_lineSlop(line_lists,m_h))

#### Tips:
画出边缘判定准确度

In [ ]:
src_main = src[y1[index]:y2[index],x1[0]:x2[-1]]
for i in range(len(new_list)):
    #print(new_list[i])
    cv2.line(src_main,(new_list[i][0][0],new_list[i][0][1]),(new_list[i][1][0],new_list[i][1][1]),(0,0,255),3)
cv2.imshow('src_main',src_main)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 05 切割图书

In [ ]:
rect_lists = fb.get_bookspine(src_main,edge_main,new_list)

### 保存结果

In [ ]:
import os
new_dir = filename + '_zero'
os.mkdir(new_dir)
for i in range(len(rect_lists)):
    wirtename = './' + new_dir + '/' + 'img_' + str(i) + '.jpg'
    cv2.imwrite(wirtename,rect_lists[i])

In [ ]:
cv2.imwrite('temp.jpg',rect_lists[0])

### Tips:

保存数据

In [ ]:
from numpy import *
savetxt('edge_1.csv',edge_main,fmt='%d',delimiter=',')
cv2.imwrite('src_1.jpg',src_main)
cv2.imwrite('edge_1.jpg',edge_main)

In [ ]:
new_list